### Step 1: Setup local configuration

Configure workspace and the resource group

Login if necessary:

In [ ]:
#!az login

Set your default Workspace and Resource Group:

In [ ]:
!az configure --defaults workspace=<workspace name> group=<resource group name>

### Step 2: Create AML environment to capture Graph UFS python dependencies

#### Generate environment configuration yaml

In [ ]:
%%writefile anemoihouse_env.yaml
$schema: https://azuremlschemas.azureedge.net/latest/environment.schema.json
name: anemoihouse
build:
  path: environment

#### Create the environment using CLI

In [ ]:
!az ml environment create --file anemoihouse_env.yaml

### Step 3: Start the training job

In [ ]:
%%writefile anemoi_training_job.yaml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json
code: ./replay/atmosphere-subsampled/p0/training
command: >-
  echo ${{inputs.data}} ${{outputs.output_dir}}; ./submit_training_aml.sh ${{inputs.data}} ${{outputs.output_dir}};
environment: azureml:anemoihouse@latest 
resources:
  instance_count: <number of VMs in a cluster(e.g.: 1,2,3,...)>
inputs:
  data: 
    type: uri_folder
    mode: ro_mount
    path: azureml:<AzureML data asset name>:<data asset version>
outputs:
  output_dir: 
    type: uri_folder
    mode: upload

distribution:
  type: mpi
  process_count_per_instance: <number of GPUs/VM>
services:
    my_vs_code:
      type: vs_code
    my_jupyter_lab:
      type: jupyter_lab
compute: azureml:<cluster name>
display_name: anemoitraining
experiment_name: anemoi
description: anemoi training job


In [ ]:
!az ml job create -f anemoi_training_job.yaml